In [235]:
import sys
import mysql.connector
import json
from pprint import pprint

def json_read(filename):
   with open(filename) as f_in:
       return(json.load(f_in))
    
structure = json_read("database_struct.json")

pprint(structure)

def connect(user):
    conn = mysql.connector.connect(host='localhost', database='microshop', user=user)
    conn.autocommit = True
    return conn

rootconn = connect('user')


def sqlQuery(sqlString, conn=rootconn):
    try:
        cursor = rootconn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchwarnings()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

{'entities': [{'Customer': {'name': 'String', 'orders': '*Order'}},
              {'Order': {'customer': 'Customer',
                         'date': 'String',
                         'lines': '*OrderLine',
                         'total': 'Number'}},
              {'OrderLine': {'count': 'Number',
                             'order': 'Order',
                             'product': 'Product',
                             'total': 'Number'}},
              {'Product': {'name': 'String', 'price': 'Number'}}],
 'schemaName': 'MicroShop'}


In [2]:
sqlQuery("show tables")

[('customer',), ('order',), ('orderline',), ('product',)]

In [240]:
sqlQuery("USE `microshop`")
sqlQuery("INSERT INTO `customer` (`name`) VALUES ('Joe');")
sqlQuery("INSERT INTO `customer` (`name`) VALUES ('Vlad');")
sqlQuery("INSERT INTO `customer` (`name`) VALUES ('Farkas');")
sqlQuery("INSERT INTO `customer` (`name`) VALUES ('Arkadius');")
sqlQuery("INSERT INTO `microshop`.`order` (`date`, `total`, `customer_id`) VALUES ('2019-01-01', '100', '6');
")
sqlQuery("INSERT INTO `microshop`.`product` (`name`, `price`) VALUES ('Mug', '99');
")
sqlQuery("INSERT INTO `microshop`.`orderline` (`order_id`, `product_id`, `count`, `total`) VALUES ('1', '1', '2', '198');
")

No result set to fetch from.
No result set to fetch from.
No result set to fetch from.
No result set to fetch from.
No result set to fetch from.


In [4]:
def myQuery(query):
    q = f"select * from {query}"
    print(q)
    return sqlQuery(q)



In [5]:
#return all customers

customers = myQuery("customer")
for customer in customers:
    print(customer)

select * from customer
(1, 'Bob')
(2, 'Dave')
(3, 'Farkas')
(4, 'Arkadius')
(5, 'Vlad')
(6, 'Joe')
(7, 'Doe')


In [6]:
#Updating myQuery

def myQuery1(query):
    res = query.split('.')
    if(len(res) < 2):
        q = f"select * from {query.lower()}"
        return sqlQuery(q) 
    else:
        q = f"select {res[1].lower()} from {res[0].lower()}"
        print(q)
        return sqlQuery(q)




In [241]:
#return all customer names

names = myQuery1("Customer.name")
for name in names:
    print(name)

select name from customer
('Bob',)
('Dave',)
('Farkas',)
('Arkadius',)
('Vlad',)
('Joe',)
('Doe',)
('Joe',)
('Vlad',)
('Farkas',)
('Arkadius',)


In [7]:
# Funky idea
#(Customer|name='Joe') , can probably be extended to the rest

def myQuery2(query):
    t = query.replace('(','').replace('|', "").replace(')',"").replace("name=", " ").replace('.',' ')
    tlist = t.split()
    return sqlQuery(f"select * from {tlist[0].lower()} where name={tlist[1]}")


    
myQuery2("(Customer|name='Joe')")

[(6, 'Joe')]

In [242]:
# Ambitious attempt at complex queries
# No more time left, but would probably refactor all of this with some nice design patterns
# Then continue with making a relationship hashMap to use when generating joins (would also help with pivots)
# And maybe parsing results into the actual python classes when not specifying an attribute

def myQuery3(query):
    match = query.find("|")
    main_class = query[1:match]
#     query = query[match+1:-1]
    where_clauses = []
    prev_match = 0
    query = query[match+1:]
    
    while(query.find("|") != -1):
        match = query.find("|")
        where_clauses.append(query[prev_match:match])
        prev_match = match
        query = query[match+1:]
        
    match = query.find(")")
    where_clauses.append(query[0:match])
    query = query[match+1:]
    
    sql = "SELECT * FROM `" + main_class.lower() + "`"
    
    if(query != ""):
        joins = query[1:].split(".")
        if(joins[-1].islower()):
            sql = "SELECT `" + joins[-1] + "` FROM `" + main_class.lower() + "`"
            del joins[-1]
        
        prev_join = main_class
        
        # Make join depending on if relationship is found on the table to be joined
        for join in joins:
            for entity in structure["entities"]:
                if join in entity:
                    for attribute_name in entity[join]:
                        if(prev_join == entity[join][attribute_name]):
                            found = True
                            break
            if(found):
                sql += "\n\t INNER JOIN `" + join.lower() + "` ON `" + prev_join.lower() + "`.id = `" + join.lower() + "`." + prev_join.lower() + "_id"
                found = False
            else:
                sql += "\n\t INNER JOIN `" + join.lower() + "` ON `" + join.lower() + "`.id = `" + prev_join.lower() + "`." + join.lower() + "_id"                            
            prev_join = join    

    if(where_clauses != []):
        for clause in where_clauses:
            sql += "\n\t WHERE `" + main_class.lower() + "`." + clause + " AND"
    
    sql = sql[:-4]
    sql += ";"
    print("WHERE CLAUSES: \n", where_clauses)
    print("\n")
    print(sql)
    print(sqlQuery(sql))


myQuery3("(Customer|name='Joe').Order.OrderLine.Product")


WHERE CLAUSES: 
 ["name='Joe'"]


SELECT * FROM `customer`
	 INNER JOIN `order` ON `customer`.id = `order`.customer_id
	 INNER JOIN `orderline` ON `order`.id = `orderline`.order_id
	 INNER JOIN `product` ON `product`.id = `orderline`.product_id
	 WHERE `customer`.name='Joe';
[(6, 'Joe', 1, 'NOW()', 100, 6, 1, 1, 1, 2, 198, 1, 'Mug', 99)]
